In [32]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [50]:
# 어떤 행렬을 정의한다.

x = np.matrix([[1,5,3],[2,3,6],[3,1,7]])

# 5. LU분해

## 1) 기본 개념
### (1) 기본 개념은, 각각 소블럭의 1행과 1열을 외적하여 행렬로 전개한 값을, 원래 행렬에서 지속적으로 빼서
### (2) a11 성분으로 나누어 1행과 1열을 원래 행렬의 값과 동일하게 만들어 준 후에
### (3) 1행과 1열을 0으로 만들어주면서 지속적으로 축소해준다.

## 2) 기본 알고리즘

In [51]:
# 1열에 1행을 외적할 경우, 생성되는 행렬의 1행과 1열엔 a00성분이 중복해서 들어가게 된다.
# normalizer는 a00로 나눠줌으로서 중복성을 제거해주고, 원래 행렬에서 잔차 행렬을 뺄 때 1행 1열은 0이 되도록 만들어주는 
# 핵심 역할을 담당한다.
normalizer = 1/(x[0,0])

# 1행에 normalizer를 곱해준다.
temp_l = normalizer * x[0].T
# 1열을 임시로 저장해주고
temp_u = x.T[0]
l = normalizer * x[0].T
u = x.T[0]

print(l,u)

[[1.]
 [5.]
 [3.]] [[1 2 3]]


In [52]:
# 1행과 1열을 외적하여 n*m 잔차행렬을 새로 생성해준다.
temp = np.matmul(temp_l,temp_u).T
print(temp)

[[ 1.  5.  3.]
 [ 2. 10.  6.]
 [ 3. 15.  9.]]


In [53]:
# 원래 행렬에서 잔차행렬을 빼 1행과 1열을 0으로 만들어준다.
x = x - temp
print(x)

[[  0.   0.   0.]
 [  0.  -7.   0.]
 [  0. -14.  -2.]]


In [54]:
normalizer = 1/x[1,1]
temp_l = normalizer * x[1].T
temp_u = x.T[1]

#위 iteration의 반복이나, i=2부터는 하삼각행렬과 상삼각행렬을 numpy의 stack으로 지속적으로 누적해준다.
l = np.hstack([l,temp_l])
u = np.vstack([u,temp_u])
temp = np.matmul(temp_l,temp_u).T
x = x - temp

In [55]:
normalizer = 1/x[2,2]
temp_l = normalizer * x[2].T
temp_u = x.T[2]
l = np.hstack([l,temp_l])
u = np.vstack([u,temp_u])
temp = np.matmul(temp_l,temp_u).T
x = x - temp
# 최종적인 잔차행렬이 0행렬이 되면 성공이다.
print(x)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [243]:
# 함수로 정의한 LU분해
def LUdecomp(x) :
    if type(x) != type(np.matrix([0])):
        raise ValueError 
    
    #행렬의 행과 열 차원중 작은쪽을 s로 지정해준다.
    s = np.min(x.shape)
    
    normalizer = 1/(x[0,0])
    l = normalizer * x[0].T
    u = x.T[0]
    temp = np.matmul(l,u).T
    x = x - temp
    
    for i in range(1,s):
        normalizer = 1/x[i,i]
        temp_l = normalizer * x[i].T
        temp_u = x.T[i]
        l = np.hstack([l,temp_l])
        u = np.vstack([u,temp_u])
        temp = np.matmul(temp_l,temp_u).T
        x = x - temp
    return (l,u)

In [58]:
x = np.matrix([[1,5,3],[2,3,6],[3,1,7]])

L,U = LUdecomp(x)

print("하삼각행렬")
print(L)
print("상삼각행렬")
print(U)

하삼각행렬
[[ 1. -0. -0.]
 [ 5.  1. -0.]
 [ 3. -0.  1.]]
상삼각행렬
[[  1.   2.   3.]
 [  0.  -7. -14.]
 [  0.   0.  -2.]]


## 3) a00 = 0일때의 대처법(피보팅)

### (1) a00 = 0일때의 문제는, 1/a00를 곱하는 과정에서 1/0이 되기 때문에 연산이 불가능해지는 점이다.
### (2) 이를 해결하기 위해, 0이 아닌 다른 행으로 교체한 후, 앞전 연산의 결과도 동일한 순서로 행교환을 해준다.
### (3) 알고리즘적으로는, 치환된 순서를 규정하는 '치환행렬'을 만들어 이를 계산에 활용한다.

- Ax = y에서 LUx = y로 분해했을 경우, 치환행렬 P를 이용하여 이를 다시 나타내면 PLUx = y, 
- 이때, P는 정규직교행렬이므로 inverse(P) = transpose(P)가 성립되고. 우변으로 이항하면 LUx = transpose(T) * y 즉 y쪽의 순서를 변경한다.
- Sxy를 i번째 iteration의 x행과 y행의 교환을 나타내는 치환행렬이라고 할 때
- P = Sxy1 * Sxy2 * ..... 이다.

In [338]:
x = np.matrix([[0,5,3,1],[2,3,6,5],[3,1,7,3],[2,1,5,6]])
print(x)

[[0 5 3 1]
 [2 3 6 5]
 [3 1 7 3]
 [2 1 5 6]]


In [245]:
LUdecomp(x) 

C:\Users\never\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in long_scalars
  if __name__ == '__main__':
C:\Users\never\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in multiply
  # Remove the CWD from sys.path while we load stuff.
C:\Users\never\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in multiply


(matrix([[nan, nan, nan, nan],
         [inf, nan, nan, nan],
         [inf, nan, nan, nan],
         [inf, nan, nan, nan]]), matrix([[  0.,   2.,   3.,   2.],
         [ nan, -inf, -inf, -inf],
         [ nan,  nan,  nan,  nan],
         [ nan,  nan,  nan,  nan]]))

- a00성분이 0이기 때문에, 무한으로 발산(inf)한다.

### (4) 핵심요소는 다음과 같다.

- 대각성분이 1인 피봇행렬을 생성한다.

In [354]:

# 그리고, 행 교환을 원하는 행의 '피봇 행렬의 행'을 서로 바꿔준다. 아래에서 그 예를 확인할 수 있다.

print(x)

pivot_table = np.matrix([[0,1,0,0],[1,0,0,0],[0,0,1,0],[0,0,0,1]]) #1행과 2행의 순서가 바뀐 pivottable을 만든다.

print(np.dot(pivot_table,x))# 2행과 1행의 순서가 바뀐것을 확인할 수 있다.

[[0 5 3 1]
 [2 3 6 5]
 [3 1 7 3]
 [2 1 5 6]]
[[2 3 6 5]
 [0 5 3 1]
 [3 1 7 3]
 [2 1 5 6]]


- 0이 아닌 행을 찾는다.


In [356]:
# X의 전치 행렬에서, 0이 아닌 가장 가까운 행을 np.where로 찾는다.
ind = np.where(x.T[0] != 0)[1][0]
print(ind) # 해당 잔차블록행렬의 첫 번째 원소가 0이 아님을 보여준다.

1


In [178]:
def making_pivot(x):
    rows = np.shape(x)[0]
    pivot_table = np.diag([1 for i in range(0,rows)])
    return pivot_table

In [179]:
def pivoting(pivot_table,where,ind):
    temp = pivot_table[where].copy()
    pivot_table[where] = pivot_table[ind].copy()
    pivot_table[ind] = temp.copy()
    return pivot_table

In [347]:
# 함수로 정의한 LU분해
def LUdecomp_pivot(x) :
    if type(x) != type(np.matrix([0])):
        raise ValueError 
    
    #행렬의 행과 열 차원중 작은쪽을 s로 지정해준다.
    s = np.min(x.shape)
    
    # X의 전치행렬에서 0이 아닌 가장 가까운 행을 np.where로 찾는다.
    ind = np.where(x.T[0] != 0)[1][0]
    # 행의 순서 교환을 기록한 pivot_table을 생성한다.
    pivot_table = pivoting(making_pivot(x),0,ind)
    x = np.matmul(pivot_table,x)
    pivot_table_summary = pivot_table.copy()
    
    normalizer = 1/(x[0,0])
    l = normalizer * x[0].T
    u = x.T[0]
    temp = np.matmul(l,u).T
    x = np.matrix(np.round(x - temp,decimals = 5))
    
    for i in range(1,s):
        # X의 전치행렬에서 0이 아닌 가장 가까운 행을 np.where로 찾는다.
        ind = np.where(x.T[i] != 0)[1][0]
        # 행의 순서 교환을 기록한 pivot_table을 생성한다.
        pivot_table = pivoting(making_pivot(x),i,ind)
        x = np.matmul(pivot_table,x)
        pivot_table_summary = np.matmul(pivot_table_summary,pivot_table)
        
        normalizer = 1/x[i,i]
        temp_l = normalizer * x[i].T
        temp_u = x.T[i]
        l = np.hstack([l,temp_l])
        u = np.vstack([u,temp_u])
        temp = np.matmul(temp_l,temp_u).T
        x = np.matrix(np.round(x - temp,decimals = 5))
    return (l,u,pivot_table_summary)

In [348]:
l,u,pivot = LUdecomp_pivot(x)

In [351]:
x

matrix([[0, 5, 3, 1],
        [2, 3, 6, 5],
        [3, 1, 7, 3],
        [2, 1, 5, 6]])

In [352]:
LU = np.matmul(l,u).T

### 피보팅된 LU분해가 끝난후, 원 행렬과 비교하면 원래와는 달리 순서가 뒤바뀌어 있음을 확인할 수 있다.

In [353]:
np.matmul(pivot,LU)

matrix([[0., 5., 3., 1.],
        [2., 3., 6., 5.],
        [3., 1., 7., 3.],
        [2., 1., 5., 6.]])

### 피보팅 행렬과 LU분해된 행렬을 곱하면, 원래 행렬이 비로소 복원됨을 확인할 수 있다.